In [ ]:



# coding: utf-8

# ### Purpose of the notebook:
# In this notebook, you'll see a simple demo of implementing Principle Component Analysis -- a dimensionality reduction method that works by creating a projection on a smaller dimension which minimizes the distance between actual features and the new projection. It is a popular method for visualization, feature generation and unsupervised clustering.
# 
# 

# ### 1. Import packages:

# In[3]:

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
matplotlib.style.use('ggplot')
get_ipython().run_line_magic('matplotlib', 'inline')


# ### 2. Generate data:
# For this first exercise, we will start with simulated data defined by a 2-dimensional Normal (Gaussian) distribution with mean vector (0,0) and covariance matrix [[3,1],[1,2]]. 

# In[148]:

# Define mean vector and covariance matrix:

fig=plt.figure(figsize=(8,6))

mu = np.zeros(2)

cov = np.array([[3,1],[1,2]])

sample=1000
data = np.random.multivariate_normal(mu,cov,size=sample)

plt.scatter(data[:,0], data[:,1])
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Simulated data (2-dimensional Gaussian)')
plt.show()


# ### 3. Implement PCA by Eigenvalue decomposition:
# 
# Essentially, PCA provides a transformed set of features such that the transformed numbers and the original features are as close as possible. One way to do that is to apply an eigen-value decomposition (which is equivalent in this case to a SVD) on the covariance matrix of a data set, so that all variables are transformed to be othogonal of each other.
# 
# Since this simulated data already comes with a covariance matrix, we can use `np.linalg.eig()` to get the eigenvalues and eigenvectors and use those to transform our original data into PC's
# 
# 

# In[150]:

# eigen-value decomposition of the covariance matrix
[S,W_true]=np.linalg.eig(cov)

fig=plt.figure(figsize=(8,6))

plt.scatter(data[:,0],data[:,1])
# Define transformed features:
z1=(W_true[0,0]/W_true[0,1])*data[:,0]
z2=(W_true[1,0]/W_true[1,1])*data[:,0]
z=np.array([z1,z2]).transpose()

plt.plot(data[:,0], z1,color='g')
plt.plot(data[:,0], z2,color='g')

g_patch = mpatches.Patch(color='g', label='Eigen-vectors (true PC components)')

plt.legend(handles=[g_patch])
plt.axis('equal')

limits = [np.minimum(np.amin(data[:,0]), np.amin(data[:,1])),
          np.maximum(np.amax(data[:,0]), np.amax(data[:,1]))]

plt.xlim(limits[0],limits[1])
plt.ylim(limits[0],limits[1])
plt.draw()


# ### 4. Define function to plot principal components

# In[183]:

from sklearn.decomposition import PCA

def plot_pc(data, pca, scatter=True, legend=True,g_patch=None):
    #fig=plt.figure(figsize=(8,6))
    # Extract weight matrix from PCA model
    W_pca = pca.components_
    if scatter:
        
        plt.scatter(data[:,0], data[:,1])
        
    plt.plot(data[:,0], -(W_pca[0,0]/W_pca[0,1])*data[:,0], color="c")
    
    plt.plot(data[:,0], -(W_pca[1,0]/W_pca[1,1])*data[:,0], color="c")
    
    if legend:
        
        c_patch = mpatches.Patch(color='c', label='Principal components')
        
        plt.legend(handles=[c_patch,g_patch], loc='lower right')
    
    plt.axis('equal')
    
    
    limits = [np.minimum(np.amin(data[:,0]), np.amin(data[:,1]))-0.5,
              np.maximum(np.amax(data[:,0]), np.amax(data[:,1]))+0.5]
    
    plt.xlim(limits[0],limits[1])
    plt.ylim(limits[0],limits[1])
    plt.show()


# ### 5. Compare results using PCA vs using eigen-value decomposition:

# In[184]:

pca = PCA(n_components=2)
pca.fit(data)

plt.scatter(data[:,0], data[:,1])

plt.plot(data[:,0], z1, color="g")
plt.plot(data[:,0], z2, color="g")

plot_pc(data, pca, scatter=True, legend=True,g_patch=g_patch)


# ### Exercise: try PCA with the Iris dataset
# 
# sklearn has some built-in data sets, let's play around with the Iris dataset, which consists of 3 different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length, stored in a 150x4 numpy.ndarray
# 
# The rows are the samples and the columns are: Sepal Length, Sepal Width, Petal Length and Petal Width, so this is a 4-dimensional data set with more features than we can visualize by eye.

# In[209]:

## Load iris data:
iris = datasets.load_iris()
data = iris.data
target = iris.target
target_names = iris.target_names


from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(1, figsize=(8, 6))

ax = Axes3D(fig, elev=-150, azim=110)

xdata = data[:,0]
ydata = data[:,1]
zdata = data[:,2]

ax.scatter3D(xdata, ydata, zdata, c=target,cmap=plt.cm.RdYlGn,edgecolor='k')


ax.set_xlabel('Sepal Length')

ax.set_ylabel('Sepal Width')

ax.set_zlabel('Petal Length')

ax.set_title('Visualizing Sepal Length, Sepal Width and Petal Length')


# In[189]:

from sklearn import datasets

def plot_iris(transformed_data, target, target_names):
    plt.figure(figsize=(8, 6))
    ## or each target, plot the data with corresponding target label and color (r, g, or b)
    for c, i, target_name in zip("ryg", [0, 1, 2], target_names):
        plt.scatter(transformed_data[target == i, 0],
                    transformed_data[target == i, 1], c=c, label=target_name)
    plt.xlabel('PC 1')
    plt.ylabel('PC 2')
    
    plt.legend()
    plt.show()
    
def write_answer(list_pc1, list_pc2):
    with open("pca_answer3.txt", "w") as fout:
        fout.write(" ".join([str(num) for num in list_pc1]))
        fout.write(" ")
        fout.write(" ".join([str(num) for num in list_pc2]))


# ### Dimensionality reduction:
# 
# Using PCA, we can reduce the dimension of our feature space from 4 to 2 without losing too much information. Below is a demo:

# In[194]:

# Create a PCA instance 2 dimensions:
pca = PCA(n_components=2)

# Fit the PCA instance with the iris data: 

princomp= pca.fit_transform(data)

plot_iris(transformed_data=princomp,target=target,target_names=target_names)


# ### Let's take a look at the principle components:
# 
# sklearn's PCA has an attribute called `explained_variance_ratio_` that tells us how much of the data's variation is explained by the chosen PC's. Essentially, that is the threshold we can use to measure against the projection error between original features and transformed features

# In[253]:


sum(pca.explained_variance_ratio_)


# In[247]:

def iris_label(x):
    if x == 0:
        lab='setosa'
    elif x == 1:
        lab = 'versicolor'
    elif x == 2:
        lab = 'virginica'
    else:
        lab = 'other'
    return lab

princomp_df=pd.DataFrame(data=princomp,columns=['PC1','PC2'])

iris_df=pd.DataFrame(data=iris.data)
iris_df.columns=['Sepal Length','Sepal Width','Petal Length','Petal Width']
iris_target=pd.DataFrame(data=iris.target)
iris_df['Label']=iris_target

iris_df['Label']=iris_df['Label'].apply(lambda s : iris_label(s))


# In[248]:

import seaborn as sns

sns.pairplot(iris_df,hue='Label')